In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_groq import ChatGroq
messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you")
]
model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=os.getenv("GROQ_API_KEY"))
result=model.invoke(messages)

In [9]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi my nameis sachchida, I am a AI enthusiast working at SAP")])

AIMessage(content="Hello Sachchida, nice to meet you. It's great to hear that you're an AI enthusiast working at SAP. SAP is a leading enterprise software company, and I'm sure you're working on some exciting projects that involve the integration of AI and machine learning.\n\nAs an AI enthusiast, you must be aware of the latest advancements in the field. I'd love to hear more about your work and projects. Are you involved in any specific areas such as natural language processing, computer vision, or predictive analytics?\n\nFeel free to share your experiences and insights, and I'll do my best to provide helpful information and engage in a conversation. What's on your mind, Sachchida?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 139, 'prompt_tokens': 51, 'total_tokens': 190, 'completion_time': 0.209537432, 'prompt_time': 0.003199247, 'queue_time': 0.044924463, 'total_time': 0.212736679}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_

In [10]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi I am Sachchida, I am working in SAP Labs in bangalore."),
        AIMessage(content="Nice to meet you, Sachchida. It's great to know that you're an AI enthusiast working at SAP. SAP is a renowned company in the enterprise software space, and AI is a rapidly evolving field with many exciting applications.\n\nWhat specific areas of AI are you interested in or working on at SAP? Are you involved in developing AI-powered solutions for customers, or are you working on research and development of new AI technologies?"),
        HumanMessage(content="Whats my name and where I am working.")
    ]
)

AIMessage(content='Your name is Sachchida, and you are working at SAP Labs in Bangalore.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 157, 'total_tokens': 175, 'completion_time': 0.029176298, 'prompt_time': 0.011149145, 'queue_time': 0.045639555, 'total_time': 0.040325443}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e556abcd-3e0f-4aa1-aefa-c70776e88509-0', usage_metadata={'input_tokens': 157, 'output_tokens': 18, 'total_tokens': 175})

#### Message History
We can use a message history class to wrap our model and make it stateful. This will keep track of input and output of the model and store them in some datastore. future interactions will then load those messages and pass them into the chain as par tof the input.

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [15]:
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(model,get_session_history=get_session_history)

In [14]:
config={"configurable":{"session_id":"chat1"}}


In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi I am Sachchida, I am an AI Engineer.")],config=config
)
response.content

'Nice to meet you again Sachchida. I see you repeated your introduction. Are you looking to discuss something specific or just wanting to start a conversation about AI and its applications?'

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="what is my name")],config=config
)
response.content

"Your name is Sachchida, and you're an AI Engineer."

### change the config

In [19]:
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="what is my name")],config=config1
)
response.content

"I don't have any information about your name as our conversation has just started. You can tell me your name if you'd like, or we can explore a different topic."

In [23]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","you are helpful assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model
chain.invoke({"messages":[HumanMessage(content="Hi My name is Sachchida")]})

AIMessage(content="Nice to meet you, Sachchida! It's great to have you here. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 57, 'total_tokens': 90, 'completion_time': 0.050409758, 'prompt_time': 0.003493282, 'queue_time': 0.044591884, 'total_time': 0.05390304}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ca59ff17-1378-4be1-bbad-a796a4af1c08-0', usage_metadata={'input_tokens': 57, 'output_tokens': 33, 'total_tokens': 90})